In [61]:
## Original code 

import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import pandas as pd
import json
import os
from datetime import datetime
import cv2
import numpy as np
import io
import re
from pathlib import Path

class PDFBillProcessor:
    def __init__(self):
        self.bill_patterns = {
            'gstin': r'GSTIN\s*:\s*([0-9A-Z]+)',
            'date': r'Date\s*:\s*(\d{2}[-/.]\d{2}[-/.]\d{4})',
            'vehicle_no': r'Vehicle\s*No\.?\s*:?\s*([A-Z]{2}[0-9]{2}[A-Z]?[0-9]{4})',
            'invoice_no': r'No\.\s*:?\s*(\d+)',
            'total_meters': r'Total\s*Metre?\s*:?\s*(\d+\.?\d*)',
        }

    def extract_pages_from_pdf(self, pdf_path):
        """Extract each page from PDF as an image"""
        doc = fitz.open(pdf_path)
        pages_data = []
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            
            # Get page as image
            pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))  # 300 DPI
            img_data = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # Convert to OpenCV format
            opencv_img = cv2.cvtColor(np.array(img_data), cv2.COLOR_RGB2BGR)
            
            pages_data.append({
                'page_num': page_num + 1,
                'image': opencv_img
            })
            
        doc.close()
        return pages_data

    def preprocess_image(self, image):
        """Enhance image for better OCR results"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding
        gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        
        # Noise removal
        gray = cv2.medianBlur(gray, 3)
        
        return gray

    def extract_bill_data(self, image):
        """Extract data from a single bill image"""
        # Preprocess image
        processed_img = self.preprocess_image(image)
        
        # Extract text using OCR
        text = pytesseract.image_to_string(processed_img)
        
        # Parse the text to extract structured data
        return self.parse_bill_text(text)

    def parse_bill_text(self, text):
        """Parse extracted text into structured data"""
        bill_data = {
            'vendor_name': '',
            'gstin': '',
            'date': '',
            'invoice_no': '',
            'vehicle_no': '',
            'items': [],
            'total_meters': 0,
            'raw_text': text  # Store raw text for verification
        }
        
        # Extract basic fields using patterns
        for field, pattern in self.bill_patterns.items():
            match = re.search(pattern, text)
            if match:
                bill_data[field] = match.group(1)
        
        # Extract vendor name (usually in the header)
        lines = text.split('\n')
        for line in lines[:5]:  # Check first 5 lines
            if 'TEXTILES' in line or 'FABRICS' in line:
                bill_data['vendor_name'] = line.strip()
                break
        
        # Extract items data
        items = []
        current_item = {}
        
        for line in lines:
            # Pattern matching for item entries
            # Adjust these patterns based on your bill format
            if re.match(r'^\d+\s+\d+\.?\d*\s+\d+\.?\d*', line):
                parts = line.split()
                items.append({
                    'serial_no': parts[0],
                    'quantity': parts[1],
                    'meters': parts[2],
                    'rate': parts[3] if len(parts) > 3 else ''
                })
        
        bill_data['items'] = items
        
        return bill_data

class BillDataFormatter:
    def __init__(self):
        self.processor = PDFBillProcessor()
        
    def process_pdf(self, pdf_path):
        """Process multi-page PDF containing bills"""
        # Extract pages
        pages_data = self.processor.extract_pages_from_pdf(pdf_path)
        
        # Process each page
        all_bills_data = []
        for page in pages_data:
            try:
                bill_data = self.processor.extract_bill_data(page['image'])
                bill_data['page_num'] = page['page_num']
                all_bills_data.append(bill_data)
            except Exception as e:
                print(f"Error processing page {page['page_num']}: {str(e)}")
        
        return all_bills_data
    
    def create_excel(self, bills_data, output_path):
        """Generate formatted Excel file"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        excel_path = os.path.join(output_path, f'bills_data_{timestamp}.xlsx')
        
        with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
            # Create Summary sheet
            summary_data = [{
                'Page': bill['page_num'],
                'Vendor': bill['vendor_name'],
                'GSTIN': bill['gstin'],
                'Date': bill['date'],
                'Invoice No': bill['invoice_no'],
                'Vehicle No': bill['vehicle_no'],
                'Total Meters': bill['total_meters']
            } for bill in bills_data]
            
            df_summary = pd.DataFrame(summary_data)
            df_summary.to_excel(writer, sheet_name='Summary', index=False)
            
            # Create Items sheet
            items_data = []
            for bill in bills_data:
                for item in bill['items']:
                    items_data.append({
                        'Page': bill['page_num'],
                        'Invoice No': bill['invoice_no'],
                        'Serial No': item.get('serial_no', ''),
                        'Quantity': item.get('quantity', ''),
                        'Meters': item.get('meters', ''),
                        'Rate': item.get('rate', '')
                    })
            
            df_items = pd.DataFrame(items_data)
            df_items.to_excel(writer, sheet_name='Items', index=False)
            
            # Format Excel
            workbook = writer.book
            header_format = workbook.add_format({
                'bold': True,
                'bg_color': '#D3D3D3',
                'border': 1
            })
            
            # Apply formatting to both sheets
            for sheet_name in ['Summary', 'Items']:
                worksheet = writer.sheets[sheet_name]
                for col_num, value in enumerate(df_summary.columns if sheet_name == 'Summary' else df_items.columns):
                    worksheet.write(0, col_num, value, header_format)
                    worksheet.set_column(col_num, col_num, 15)
            
        return excel_path

    def create_json(self, bills_data, output_path):
        """Generate JSON file"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        json_path = os.path.join(output_path, f'bills_data_{timestamp}.json')
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(bills_data, f, indent=2, ensure_ascii=False)
        
        return json_path

def process_pdf_bills(pdf_path, output_path, output_format='excel'):
    """Main function to process PDF bills"""
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize formatter
    formatter = BillDataFormatter()
    
    # Process PDF
    bills_data = formatter.process_pdf(pdf_path)
    
    # Generate output file
    if output_format.lower() == 'excel':
        output_file = formatter.create_excel(bills_data, output_path)
    else:
        output_file = formatter.create_json(bills_data, output_path)
    
    return output_file

# Example usage
if __name__ == "__main__":
    # Configuration
    pdf_path = "ocrr.pdf"
    # output_folder = "./output"
    output_folder = "C:/Users/ranja/Downloads"
    output_format = "excel"  # or "json"
    
    try:
        output_file = process_pdf_bills(pdf_path, output_folder, output_format)
        print(f"Successfully processed bills. Output saved to: {output_file}")
    except Exception as e:
        print(f"Error processing bills: {str(e)}")

Successfully processed bills. Output saved to: C:/Users/ranja/Downloads\bills_data_20241109_200948.xlsx


In [66]:
#displys all vendors names

import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import pandas as pd
import json
import os
from datetime import datetime
import cv2
import numpy as np
import io
import re
from pathlib import Path

class PDFBillProcessor:
    def __init__(self):
        self.bill_patterns = {
            'gstin': r'GSTIN\s*:\s*([0-9A-Z]+)',
            'date': r'Date\s*:\s*(\d{2}[-/.]\d{2}[-/.]\d{4})',
            'vehicle_no': r'Vehicle\s*No\.?\s*:?\s*([A-Z]{2}[0-9]{2}[A-Z]?[0-9]{4})',
            'invoice_no': r'No\.\s*:?\s*(\d+)',
            'total_meters': r'Total\s*Metre?\s*:?\s*(\d+\.?\d*)',
        }

    def extract_pages_from_pdf(self, pdf_path):
        """Extract each page from PDF as an image"""
        doc = fitz.open(pdf_path)
        pages_data = []
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            
            # Get page as image
            pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))  # 300 DPI
            img_data = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # Convert to OpenCV format
            opencv_img = cv2.cvtColor(np.array(img_data), cv2.COLOR_RGB2BGR)
            
            pages_data.append({
                'page_num': page_num + 1,
                'image': opencv_img
            })
            
        doc.close()
        return pages_data

    def preprocess_image(self, image):
        """Enhance image for better OCR results"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding
        gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        
        # Noise removal
        gray = cv2.medianBlur(gray, 3)
        
        return gray

    def extract_bill_data(self, image):
        """Extract data from a single bill image"""
        # Preprocess image
        processed_img = self.preprocess_image(image)
        
        # Extract text using OCR
        text = pytesseract.image_to_string(processed_img)
        
        # Parse the text to extract structured data
        return self.parse_bill_text(text)

    def parse_bill_text(self, text):
        """Parse extracted text into structured data"""
        bill_data = {
            'vendor_name': '',
            'gstin': '',
            'date': '',
            'invoice_no': '',
            'vehicle_no': '',
            'items': [],
            'total_meters': 0,
            'raw_text': text  # Store raw text for verification
        }
        
        # Extract basic fields using patterns
        for field, pattern in self.bill_patterns.items():
            match = re.search(pattern, text)
            if match:
                bill_data[field] = match.group(1)
        
        # Extract vendor name (both left-aligned and center-aligned)
        lines = text.split('\n')
        
        # Check for vendor names that might be left-aligned or center-aligned
        for line in lines[:5]:  # Check first 5 lines for vendor name
            if 'TEXTILES' in line or 'FABRICS' in line:
                bill_data['vendor_name'] = line.strip()
                break
            # Additional check for left-aligned vendor names (if no match found in center)
            elif len(line.strip()) > 0 and not bill_data['vendor_name']:
                bill_data['vendor_name'] = line.strip()
    
        # Extract items data
        items = []
        current_item = {}
        
        for line in lines:
            # Pattern matching for item entries
            # Adjust these patterns based on your bill format
            if re.match(r'^\d+\s+\d+\.?\d*\s+\d+\.?\d*', line):
                parts = line.split()
                items.append({
                    'serial_no': parts[0],
                    'quantity': parts[1],
                    'meters': parts[2],
                    'rate': parts[3] if len(parts) > 3 else ''
                })
        
        bill_data['items'] = items
        
        return bill_data


class BillDataFormatter:
    def __init__(self):
        self.processor = PDFBillProcessor()
        
    def process_pdf(self, pdf_path):
        """Process multi-page PDF containing bills"""
        # Extract pages
        pages_data = self.processor.extract_pages_from_pdf(pdf_path)
        
        # Process each page
        all_bills_data = []
        for page in pages_data:
            try:
                bill_data = self.processor.extract_bill_data(page['image'])
                bill_data['page_num'] = page['page_num']
                all_bills_data.append(bill_data)
            except Exception as e:
                print(f"Error processing page {page['page_num']}: {str(e)}")
        
        return all_bills_data
    
    def create_excel(self, bills_data, output_path):
        """Generate formatted Excel file"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        excel_path = os.path.join(output_path, f'bills_data_{timestamp}.xlsx')
        
        with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
            # Create Summary sheet
            summary_data = [{
                'Page': bill['page_num'],
                'Vendor': bill['vendor_name'],
                'GSTIN': bill['gstin'],
                'Date': bill['date'],
                'Invoice No': bill['invoice_no'],
                'Vehicle No': bill['vehicle_no'],
                'Total Meters': bill['total_meters']
            } for bill in bills_data]
            
            df_summary = pd.DataFrame(summary_data)
            df_summary.to_excel(writer, sheet_name='Summary', index=False)
            
            # Create Items sheet
            items_data = []
            for bill in bills_data:
                for item in bill['items']:
                    items_data.append({
                        'Page': bill['page_num'],
                        'Invoice No': bill['invoice_no'],
                        'Serial No': item.get('serial_no', ''),
                        'Quantity': item.get('quantity', ''),
                        'Meters': item.get('meters', ''),
                        'Rate': item.get('rate', '')
                    })
            
            df_items = pd.DataFrame(items_data)
            df_items.to_excel(writer, sheet_name='Items', index=False)
            
            # Format Excel
            workbook = writer.book
            header_format = workbook.add_format({
                'bold': True,
                'bg_color': '#D3D3D3',
                'border': 1
            })
            
            # Apply formatting to both sheets
            for sheet_name in ['Summary', 'Items']:
                worksheet = writer.sheets[sheet_name]
                for col_num, value in enumerate(df_summary.columns if sheet_name == 'Summary' else df_items.columns):
                    worksheet.write(0, col_num, value, header_format)
                    worksheet.set_column(col_num, col_num, 15)
            
        return excel_path

    def create_json(self, bills_data, output_path):
        """Generate JSON file"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        json_path = os.path.join(output_path, f'bills_data_{timestamp}.json')
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(bills_data, f, indent=2, ensure_ascii=False)
        
        return json_path

def process_pdf_bills(pdf_path, output_path, output_format='excel'):
    """Main function to process PDF bills"""
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize formatter
    formatter = BillDataFormatter()
    
    # Process PDF
    bills_data = formatter.process_pdf(pdf_path)
    
    # Generate output file
    if output_format.lower() == 'excel':
        output_file = formatter.create_excel(bills_data, output_path)
    else:
        output_file = formatter.create_json(bills_data, output_path)
    
    return output_file

# Example usage
if __name__ == "__main__":
    # Configuration
    pdf_path = "ocrr.pdf"
    # output_folder = "./output"
    output_folder = "C:/Users/ranja/Downloads"
    output_format = "excel"  # or "json"
    
    try:
        output_file = process_pdf_bills(pdf_path, output_folder, output_format)
        print(f"Successfully processed bills. Output saved to: {output_file}")
    except Exception as e:
        print(f"Error processing bills: {str(e)}")

Successfully processed bills. Output saved to: C:/Users/ranja/Downloads\bills_data_20241109_201828.xlsx


In [81]:
# dispalys Vendors name and GSTIN

import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import pandas as pd
import json
import os
from datetime import datetime
import cv2
import numpy as np
import io
import re
from pathlib import Path

class PDFBillProcessor:
    def __init__(self):
        self.bill_patterns = {
            'gstin': r'GSTIN\s*[:\-\s]*([0-9A-Z]{15})',  # Update regex to capture 15-character GSTIN
            'date': r'Date\s*:\s*(\d{2}[-/.]\d{2}[-/.]\d{4})',
            'vehicle_no': r'Vehicle\s*No\.?\s*:?\s*([A-Z]{2}[0-9]{2}[A-Z]?[0-9]{4})',
            'invoice_no': r'No\.\s*:?\s*(\d+)',
            'total_meters': r'Total\s*Metre?\s*:?\s*(\d+\.?\d*)',
        }
        # Add new regex pattern to match GSTIN anywhere in the document
        self.gstin_regex = r'\d{2}[A-Z]{5}\d{4}[A-Z]{1}\d{1}[A-Z]{1}'  # GSTIN format

    def extract_pages_from_pdf(self, pdf_path):
        """Extract each page from PDF as an image"""
        doc = fitz.open(pdf_path)
        pages_data = []
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            
            # Get page as image
            pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))  # 300 DPI
            img_data = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # Convert to OpenCV format
            opencv_img = cv2.cvtColor(np.array(img_data), cv2.COLOR_RGB2BGR)
            
            pages_data.append({
                'page_num': page_num + 1,
                'image': opencv_img
            })
            
        doc.close()
        return pages_data

    def preprocess_image(self, image):
        """Enhance image for better OCR results"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding
        gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        
        # Noise removal
        gray = cv2.medianBlur(gray, 3)
        
        return gray

    def extract_bill_data(self, image):
        """Extract data from a single bill image"""
        # Preprocess image
        processed_img = self.preprocess_image(image)
        
        # Extract text using OCR
        text = pytesseract.image_to_string(processed_img)
        
        # Parse the text to extract structured data
        return self.parse_bill_text(text)

    def parse_bill_text(self, text):
        """Parse extracted text into structured data"""
        bill_data = {
            'vendor_name': '',
            'gstin': '',  # Ensure the 'gstin' key is present
            'date': '',
            'invoice_no': '',
            'vehicle_no': '',
            'items': [],
            'total_meters': 0,
            'raw_text': text  # Store raw text for verification
        }
        
        # Extract basic fields using patterns
        for field, pattern in self.bill_patterns.items():
            match = re.search(pattern, text)
            if match:
                bill_data[field] = match.group(1)
        
        # Extract GSTIN using the new regex pattern
        gstin_match = re.search(self.gstin_regex, text)
        if gstin_match:
            bill_data['gstin'] = gstin_match.group(0)  # Store the matched GSTIN

        # Extract vendor name (both left-aligned and center-aligned)
        lines = text.split('\n')
        
        # Check for vendor names that might be left-aligned or center-aligned
        for line in lines[:5]:  # Check first 5 lines for vendor name
            if 'TEXTILES' in line or 'FABRICS' in line:
                bill_data['vendor_name'] = line.strip()
                break
            # Additional check for left-aligned vendor names (if no match found in center)
            elif len(line.strip()) > 0 and not bill_data['vendor_name']:
                bill_data['vendor_name'] = line.strip()
    
        # Extract items data
        items = []
        current_item = {}
        
        for line in lines:
            # Pattern matching for item entries
            # Adjust these patterns based on your bill format
            if re.match(r'^\d+\s+\d+\.?\d*\s+\d+\.?\d*', line):
                parts = line.split()
                items.append({
                    'serial_no': parts[0],
                    'quantity': parts[1],
                    'meters': parts[2],
                    'rate': parts[3] if len(parts) > 3 else ''
                })
        
        bill_data['items'] = items
        
        return bill_data



class BillDataFormatter:
    def __init__(self):
        self.processor = PDFBillProcessor()
        
    def process_pdf(self, pdf_path):
        """Process multi-page PDF containing bills"""
        # Extract pages
        pages_data = self.processor.extract_pages_from_pdf(pdf_path)
        
        # Process each page
        all_bills_data = []
        for page in pages_data:
            try:
                bill_data = self.processor.extract_bill_data(page['image'])
                bill_data['page_num'] = page['page_num']
                all_bills_data.append(bill_data)
            except Exception as e:
                print(f"Error processing page {page['page_num']}: {str(e)}")
        
        return all_bills_data
    
    def create_excel(self, bills_data, output_path):
        """Generate formatted Excel file"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        excel_path = os.path.join(output_path, f'bills_data_{timestamp}.xlsx')
        
        with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
            # Create Summary sheet
            summary_data = [{
                'Page': bill['page_num'],
                'Vendor': bill['vendor_name'],
                'GSTIN': bill['gstin'],  # Add GSTIN data here
                'Date': bill['date'],
                'Invoice No': bill['invoice_no'],
                'Vehicle No': bill['vehicle_no'],
                'Total Meters': bill['total_meters']
            } for bill in bills_data]
            
            df_summary = pd.DataFrame(summary_data)
            df_summary.to_excel(writer, sheet_name='Summary', index=False)
            
            # Create Items sheet
            items_data = []
            for bill in bills_data:
                for item in bill['items']:
                    items_data.append({
                        'Page': bill['page_num'],
                        'Invoice No': bill['invoice_no'],
                        'Serial No': item.get('serial_no', ''),
                        'Quantity': item.get('quantity', ''),
                        'Meters': item.get('meters', ''),
                        'Rate': item.get('rate', '')
                    })
            
            df_items = pd.DataFrame(items_data)
            df_items.to_excel(writer, sheet_name='Items', index=False)
            
            # Format Excel
            workbook = writer.book
            header_format = workbook.add_format({
                'bold': True,
                'bg_color': '#D3D3D3',
                'border': 1
            })
            
            # Apply formatting to both sheets
            for sheet_name in ['Summary', 'Items']:
                worksheet = writer.sheets[sheet_name]
                for col_num, value in enumerate(df_summary.columns if sheet_name == 'Summary' else df_items.columns):
                    worksheet.write(0, col_num, value, header_format)
                    worksheet.set_column(col_num, col_num, 15)
            
        return excel_path

    def create_json(self, bills_data, output_path):
        """Generate JSON file"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        json_path = os.path.join(output_path, f'bills_data_{timestamp}.json')
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(bills_data, f, indent=2, ensure_ascii=False)
        
        return json_path

def process_pdf_bills(pdf_path, output_path, output_format='excel'):
    """Main function to process PDF bills"""
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize formatter
    formatter = BillDataFormatter()
    
    # Process PDF
    bills_data = formatter.process_pdf(pdf_path)
    
    # Generate output file
    if output_format.lower() == 'excel':
        output_file = formatter.create_excel(bills_data, output_path)
    else:
        output_file = formatter.create_json(bills_data, output_path)
    
    return output_file

# Example usage
if __name__ == "__main__":
    # Configuration
    pdf_path = "ocrr.pdf"
    # output_folder = "./output"
    output_folder = "C:/Users/ranja/Downloads"
    output_format = "excel"  # or "json"
    
    try:
        output_file = process_pdf_bills(pdf_path, output_folder, output_format)
        print(f"Successfully processed bills. Output saved to: {output_file}")
    except Exception as e:
        print(f"Error processing bills: {str(e)}")


Successfully processed bills. Output saved to: C:/Users/ranja/Downloads\bills_data_20241109_203022.xlsx
